# Notebook 00: Permutations and Inversions

**Paper References:**
- **Logic Realism**: Section 3 (Logical Field Structure)
- **Paper I**: Section 2.1-2.2 (Permutation Representation)

**Purpose**: Establish the foundational structures of the symmetric group $S_N$, inversion count metric $h(\sigma)$, permutohedron geometry, and Cayley graph construction.

**Generates:**
- Table: Permutation enumeration for N=3,4,5 with inversion counts
- Figure: Cayley graph for $S_3$, $S_4$
- Figure: Permutohedron visualization N=3 (2D), N=4 (3D)

---

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import permutations
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality defaults
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'

# Create output directories
import os
os.makedirs('outputs/figures', exist_ok=True)
os.makedirs('outputs/tables', exist_ok=True)

print("✓ Environment setup complete")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ NetworkX version: {nx.__version__}")

## 2. Permutation Representation

### 2.1 One-Line Notation

We represent permutations in **one-line notation**: $\sigma = [\sigma(1), \sigma(2), \ldots, \sigma(N)]$

**Example**: For $N=3$, the permutation $\sigma = [2,3,1]$ means:
- $\sigma(1) = 2$
- $\sigma(2) = 3$  
- $\sigma(3) = 1$

**Identity**: $e = [1,2,3,\ldots,N]$

In [ ]:
def generate_permutations(N):
    """
    Generate all permutations of {1,2,...,N} in one-line notation.
    
    Args:
        N (int): Size of the symmetric group
        
    Returns:
        list: All N! permutations as tuples
    """
    return list(permutations(range(1, N+1)))

# Example: Generate S_3
S_3 = generate_permutations(3)
print(f"S_3 has {len(S_3)} elements:")
for i, sigma in enumerate(S_3, 1):
    print(f"  {i}. {list(sigma)}")

# Verify identity
identity_3 = tuple(range(1, 4))
print(f"\nIdentity element: {list(identity_3)}")
print(f"✓ Identity is first element: {S_3[0] == identity_3}")

## 3. Inversion Count Metric

### 3.1 Definition

The **inversion count** $h(\sigma)$ counts the number of pairs $(i,j)$ where $i < j$ but $\sigma(i) > \sigma(j)$.

**Formula**: 
$$h(\sigma) = |\{(i,j) : i < j \text{ and } \sigma(i) > \sigma(j)\}|$$

**Properties**:
- $h(e) = 0$ (identity has zero inversions)
- $0 \leq h(\sigma) \leq \binom{N}{2} = \frac{N(N-1)}{2}$
- $h$ measures "distance from sorted order"

In [ ]:
def inversion_count(sigma):
    """
    Compute the inversion count h(sigma) of a permutation.
    
    Args:
        sigma (tuple): Permutation in one-line notation
        
    Returns:
        int: Number of inversions
    """
    count = 0
    N = len(sigma)
    for i in range(N):
        for j in range(i+1, N):
            if sigma[i] > sigma[j]:
                count += 1
    return count

# Test on S_3
print("Inversion counts for S_3:")
print("="*40)
for sigma in S_3:
    h = inversion_count(sigma)
    print(f"{list(sigma):<12} → h = {h}")

# Verify properties
h_values = [inversion_count(sigma) for sigma in S_3]
print(f"\n✓ Identity has h=0: {inversion_count((1,2,3)) == 0}")
print(f"✓ Maximum h = {max(h_values)} = {3*2//2}")
print(f"✓ Range: [{min(h_values)}, {max(h_values)}]")

### 3.2 Inversion Count as a Metric

The inversion count satisfies **5 criteria** making it the natural distance measure on $S_N$:

1. **Identity**: $h(e) = 0$
2. **Non-negativity**: $h(\sigma) \geq 0$ for all $\sigma$
3. **Boundedness**: $h(\sigma) \leq \binom{N}{2}$
4. **Monotonicity**: Adjacent transpositions increase $h$ by exactly 1
5. **Mahonian symmetry**: $h$ partitions $S_N$ symmetrically

These properties are proven in **Paper I, Section 2.2**.

In [ ]:
def adjacent_transposition(sigma, i):
    """
    Apply adjacent transposition s_i to sigma (swap positions i and i+1).
    
    Args:
        sigma (tuple): Permutation
        i (int): Index (0-based) of transposition
        
    Returns:
        tuple: Resulting permutation
    """
    sigma_list = list(sigma)
    sigma_list[i], sigma_list[i+1] = sigma_list[i+1], sigma_list[i]
    return tuple(sigma_list)

# Test monotonicity property
print("Adjacent transposition effects on h:")
print("="*50)
sigma = (1, 2, 3)
h_before = inversion_count(sigma)
print(f"Start: {list(sigma):<12} h = {h_before}")

for i in range(2):
    sigma_new = adjacent_transposition(sigma, i)
    h_after = inversion_count(sigma_new)
    delta_h = h_after - h_before
    print(f"  s_{i+1}: {list(sigma_new):<12} h = {h_after} (Δh = {delta_h:+d})")

print("\n✓ Adjacent transpositions change h by ±1")

## 4. Permutation Enumeration Tables

Generate complete enumeration tables for N=3,4,5 with inversion counts.

In [ ]:
def create_permutation_table(N):
    """
    Create a DataFrame with all permutations and their inversion counts.
    
    Args:
        N (int): Size of symmetric group
        
    Returns:
        pd.DataFrame: Table with columns ['Index', 'Permutation', 'h(σ)']
    """
    S_N = generate_permutations(N)
    data = []
    for i, sigma in enumerate(S_N, 1):
        h = inversion_count(sigma)
        data.append({
            'Index': i,
            'Permutation': str(list(sigma)),
            'h(σ)': h
        })
    return pd.DataFrame(data)

# Generate tables for N=3,4,5
for N in [3, 4, 5]:
    df = create_permutation_table(N)
    print(f"\n{'='*60}")
    print(f"S_{N}: {len(df)} permutations")
    print(f"{'='*60}")
    print(df.to_string(index=False))
    
    # Save to CSV
    filename = f'outputs/tables/00_permutation_enumeration_N{N}.csv'
    df.to_csv(filename, index=False)
    print(f"\n✓ Saved to {filename}")

# Summary statistics
print(f"\n{'='*60}")
print("Summary Statistics")
print(f"{'='*60}")
for N in [3, 4, 5]:
    S_N = generate_permutations(N)
    h_vals = [inversion_count(s) for s in S_N]
    print(f"N={N}: |S_{N}| = {len(S_N):>3}, h_max = {max(h_vals):>2}, h_avg = {np.mean(h_vals):.2f}")

## 5. Cayley Graph Construction

### 5.1 Definition

The **Cayley graph** of $S_N$ with generating set $\{s_1, s_2, \ldots, s_{N-1}\}$ (adjacent transpositions) has:
- **Vertices**: All permutations in $S_N$
- **Edges**: $\sigma \to \sigma \cdot s_i$ for each generator $s_i$

**Properties**:
- Edge graph of the permutohedron
- Shortest path distance = inversion count difference (Kendall tau distance)
- $(N-1)$-regular graph (each vertex has degree $N-1$)

In [ ]:
def construct_cayley_graph(N):
    """
    Construct the Cayley graph of S_N with adjacent transposition generators.
    
    Args:
        N (int): Size of symmetric group
        
    Returns:
        nx.Graph: Cayley graph with node attributes for h(σ)
    """
    G = nx.Graph()
    S_N = generate_permutations(N)
    
    # Add nodes with inversion count attribute
    for sigma in S_N:
        h = inversion_count(sigma)
        G.add_node(sigma, h=h, label=str(list(sigma)))
    
    # Add edges for each adjacent transposition
    for sigma in S_N:
        for i in range(N-1):
            sigma_adj = adjacent_transposition(sigma, i)
            if sigma_adj in S_N:
                G.add_edge(sigma, sigma_adj, generator=f's_{i+1}')
    
    return G

# Construct for N=3
G_3 = construct_cayley_graph(3)
print(f"Cayley graph of S_3:")
print(f"  Vertices: {G_3.number_of_nodes()}")
print(f"  Edges: {G_3.number_of_edges()}")
print(f"  Regular degree: {[d for n, d in G_3.degree()][0]}")
print(f"  ✓ Graph is {3-1}-regular: {all(d == 2 for n, d in G_3.degree())}")

### 5.2 Visualization: Cayley Graph of $S_3$

In [ ]:
def plot_cayley_graph_S3(G, save=True):
    """
    Plot Cayley graph of S_3 with inversion count coloring.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Layout: arrange by h-value
    pos = {}
    h_groups = {}
    for node in G.nodes():
        h = G.nodes[node]['h']
        if h not in h_groups:
            h_groups[h] = []
        h_groups[h].append(node)
    
    # Position nodes in layers by h-value
    for h, nodes in h_groups.items():
        n_nodes = len(nodes)
        for i, node in enumerate(nodes):
            x = (i - (n_nodes-1)/2) * 1.5
            y = -h * 1.5
            pos[node] = (x, y)
    
    # Node colors by h-value
    h_values = [G.nodes[n]['h'] for n in G.nodes()]
    node_colors = plt.cm.viridis([h/max(h_values) for h in h_values])
    
    # Draw
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=2, ax=ax)
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          node_size=800, edgecolors='black', linewidths=2, ax=ax)
    
    # Labels
    labels = {n: G.nodes[n]['label'] for n in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold', ax=ax)
    
    # h-value labels
    for h in h_groups.keys():
        ax.text(-4, -h*1.5, f'h = {h}', fontsize=12, fontweight='bold',
               verticalalignment='center')
    
    ax.set_title(r'Cayley Graph of $S_3$ (Adjacent Transposition Generators)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.text(0.5, -0.02, 'Color indicates inversion count h(σ)', 
           transform=ax.transAxes, ha='center', fontsize=10, style='italic')
    ax.axis('off')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    
    if save:
        plt.savefig('outputs/figures/00_cayley_graph_S3.png', dpi=300, bbox_inches='tight')
        plt.savefig('outputs/figures/00_cayley_graph_S3.svg', bbox_inches='tight')
        print("✓ Saved Cayley graph of S_3")
    
    plt.show()

plot_cayley_graph_S3(G_3)

## 6. Permutohedron Geometry

### 6.1 Definition

The **permutohedron** $\Pi_N$ is the $(N-1)$-dimensional convex polytope with:
- **Vertices**: Points $(\sigma(1), \sigma(2), \ldots, \sigma(N))$ for each $\sigma \in S_N$
- **Edges**: Connect permutations differing by an adjacent transposition
- **Lives in**: Hyperplane $\sum x_i = \frac{N(N+1)}{2}$ in $\mathbb{R}^N$

**Dimension**: $(N-1)$ (one constraint reduces dimension by 1)

**Examples**:
- $N=3$: Hexagon in 2D plane
- $N=4$: Truncated octahedron in 3D space

In [ ]:
def permutohedron_embedding(sigma):
    """
    Embed permutation as vertex of permutohedron.
    
    Args:
        sigma (tuple): Permutation
        
    Returns:
        np.array: Coordinates in R^N
    """
    return np.array(sigma, dtype=float)

def project_to_plane(points):
    """
    Project N-dimensional points to 2D for visualization.
    Uses PCA to find best 2D projection.
    """
    # Center points
    centered = points - points.mean(axis=0)
    
    # Compute covariance and eigenvectors
    cov = np.cov(centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # Project onto top 2 principal components
    idx = eigenvalues.argsort()[::-1]
    top2 = eigenvectors[:, idx[:2]]
    projected = centered @ top2
    
    return projected

# Test embedding
S_3 = generate_permutations(3)
points_3d = np.array([permutohedron_embedding(s) for s in S_3])
print(f"Permutohedron vertices for S_3:")
print(points_3d)
print(f"\n✓ All points satisfy ∑x_i = {3*4//2}: ",
      all(abs(p.sum() - 6) < 1e-10 for p in points_3d))

### 6.2 Visualization: Permutohedron of $S_3$ (2D Hexagon)

In [ ]:
def plot_permutohedron_S3(save=True):
    """
    Plot permutohedron of S_3 as a hexagon.
    """
    S_3 = generate_permutations(3)
    G_3 = construct_cayley_graph(3)
    
    # Embed and project to 2D
    points = np.array([permutohedron_embedding(s) for s in S_3])
    points_2d = project_to_plane(points)
    
    # Create position dict
    pos = {s: points_2d[i] for i, s in enumerate(S_3)}
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Node colors by h-value
    h_values = [inversion_count(s) for s in S_3]
    node_colors = plt.cm.viridis([h/max(h_values) for h in h_values])
    
    # Draw edges
    nx.draw_networkx_edges(G_3, pos, alpha=0.4, width=3, ax=ax)
    
    # Draw nodes
    nx.draw_networkx_nodes(G_3, pos, node_color=node_colors,
                          node_size=1200, edgecolors='black', linewidths=2.5, ax=ax)
    
    # Labels
    labels = {s: str(list(s)) for s in S_3}
    nx.draw_networkx_labels(G_3, pos, labels, font_size=9, font_weight='bold', ax=ax)
    
    # h-value annotations
    for s, (x, y) in pos.items():
        h = inversion_count(s)
        offset = 0.25
        dx = x / np.linalg.norm([x, y]) * offset if np.linalg.norm([x, y]) > 0 else 0
        dy = y / np.linalg.norm([x, y]) * offset if np.linalg.norm([x, y]) > 0 else 0
        ax.text(x + dx, y + dy, f'h={h}', fontsize=8, 
               bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8),
               ha='center', va='center')
    
    ax.set_title(r'Permutohedron $\Pi_3$ (Hexagon in $\mathbb{R}^2$)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.text(0.5, -0.02, 'Vertices: all permutations of [1,2,3] | Edges: adjacent transpositions', 
           transform=ax.transAxes, ha='center', fontsize=10, style='italic')
    ax.axis('off')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    
    if save:
        plt.savefig('outputs/figures/00_permutohedron_N3.png', dpi=300, bbox_inches='tight')
        plt.savefig('outputs/figures/00_permutohedron_N3.svg', bbox_inches='tight')
        print("✓ Saved permutohedron of S_3")
    
    plt.show()

plot_permutohedron_S3()

### 6.3 Visualization: Permutohedron of $S_4$ (3D Truncated Octahedron)

In [ ]:
def plot_permutohedron_S4(save=True):
    """
    Plot permutohedron of S_4 as a 3D truncated octahedron.
    """
    S_4 = generate_permutations(4)
    G_4 = construct_cayley_graph(4)
    
    # Embed in R^4 and project to 3D
    points = np.array([permutohedron_embedding(s) for s in S_4])
    
    # Center
    centered = points - points.mean(axis=0)
    
    # Project to top 3 principal components
    cov = np.cov(centered.T)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    idx = eigenvalues.argsort()[::-1]
    top3 = eigenvectors[:, idx[:3]]
    points_3d = centered @ top3
    
    # Create position dict
    pos_3d = {s: points_3d[i] for i, s in enumerate(S_4)}
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Node colors by h-value
    h_values = [inversion_count(s) for s in S_4]
    node_colors = plt.cm.viridis([h/max(h_values) for h in h_values])
    
    # Draw edges
    for edge in G_4.edges():
        p1, p2 = pos_3d[edge[0]], pos_3d[edge[1]]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 
               'gray', alpha=0.3, linewidth=1)
    
    # Draw nodes
    for i, s in enumerate(S_4):
        p = pos_3d[s]
        ax.scatter(p[0], p[1], p[2], c=[node_colors[i]], s=150, 
                  edgecolors='black', linewidths=1.5)
    
    ax.set_title(r'Permutohedron $\Pi_4$ (Truncated Octahedron in $\mathbb{R}^3$)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.text2D(0.5, 0.02, '24 vertices (permutations of [1,2,3,4]) connected by adjacent transpositions', 
             transform=ax.transAxes, ha='center', fontsize=10, style='italic')
    
    # Remove axis
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    
    # Set viewing angle
    ax.view_init(elev=20, azim=45)
    
    plt.tight_layout()
    
    if save:
        plt.savefig('outputs/figures/00_permutohedron_N4.png', dpi=300, bbox_inches='tight')
        plt.savefig('outputs/figures/00_permutohedron_N4.svg', bbox_inches='tight')
        print("✓ Saved permutohedron of S_4")
    
    plt.show()

plot_permutohedron_S4()

## 7. Summary and Validation

### 7.1 Key Results

This notebook established:

1. **Permutation representation** in one-line notation
2. **Inversion count metric** $h(\sigma)$ satisfying 5 criteria
3. **Cayley graph structure** with adjacent transposition generators
4. **Permutohedron geometry** as $(N-1)$-dimensional convex polytope

### 7.2 Outputs Generated

**Tables**:
- Permutation enumeration for N=3,4,5 with inversion counts

**Figures**:
- Cayley graph of $S_3$
- Permutohedron $\Pi_3$ (2D hexagon)
- Permutohedron $\Pi_4$ (3D truncated octahedron)

### 7.3 Validation Checks

In [ ]:
print("="*60)
print("VALIDATION SUMMARY")
print("="*60)

validation_results = []

# Check 1: Correct cardinalities
for N in [3, 4, 5]:
    S_N = generate_permutations(N)
    expected = np.math.factorial(N)
    actual = len(S_N)
    passed = (actual == expected)
    validation_results.append(passed)
    status = "✓" if passed else "✗"
    print(f"{status} |S_{N}| = {actual} (expected {expected})")

# Check 2: Identity has h=0
for N in [3, 4, 5]:
    identity = tuple(range(1, N+1))
    h_id = inversion_count(identity)
    passed = (h_id == 0)
    validation_results.append(passed)
    status = "✓" if passed else "✗"
    print(f"{status} h(identity_{N}) = {h_id}")

# Check 3: Maximum h-value
for N in [3, 4, 5]:
    S_N = generate_permutations(N)
    h_max = max(inversion_count(s) for s in S_N)
    expected_max = N * (N-1) // 2
    passed = (h_max == expected_max)
    validation_results.append(passed)
    status = "✓" if passed else "✗"
    print(f"{status} max h(S_{N}) = {h_max} (expected {expected_max})")

# Check 4: Cayley graph regularity
for N in [3, 4]:
    G = construct_cayley_graph(N)
    degrees = [d for n, d in G.degree()]
    is_regular = all(d == N-1 for d in degrees)
    validation_results.append(is_regular)
    status = "✓" if is_regular else "✗"
    print(f"{status} Cayley graph S_{N} is {N-1}-regular")

# Check 5: Permutohedron constraint
for N in [3, 4]:
    S_N = generate_permutations(N)
    points = [permutohedron_embedding(s) for s in S_N]
    expected_sum = N * (N+1) // 2
    all_correct = all(abs(p.sum() - expected_sum) < 1e-10 for p in points)
    validation_results.append(all_correct)
    status = "✓" if all_correct else "✗"
    print(f"{status} All permutohedron vertices satisfy ∑x_i = {expected_sum} (N={N})")

# Overall
print("\n" + "="*60)
all_passed = all(validation_results)
print(f"Overall: {len([r for r in validation_results if r])}/{len(validation_results)} checks passed")
if all_passed:
    print("✓ ALL VALIDATION CHECKS PASSED")
else:
    print("✗ SOME VALIDATION CHECKS FAILED")
print("="*60)

## 8. References and Next Steps

### Paper References

- **Logic Realism Paper, Section 3**: Logical Field Structure
- **Paper I, Section 2.1-2.2**: Permutation Representation and Inversion Count

### Next Notebook

**Notebook 01: Logical Operators** will implement:
- Identity (ID), Non-Contradiction (NC), Excluded Middle (EM) operators
- Logical filtering operator $L = EM \circ NC \circ ID$
- Valid state space $V_K = \{\sigma : h(\sigma) \leq K\}$
- Visualization of $V_K$ subsets on permutohedron

---

**Notebook 00 Complete** ✓